# Spooky Author - Feature Extraction for Topic Modelling and Visualizations



**I will be doing features engineering and try to generate all possible kind of features**


 
| S. no.                 | Feature type                                                                                                   |                Status                            |
|:---------------------:|:---------------------------------------------------------------------------------------------------:|:-------------------------------------------:|
| 1                         | Author characteristic writing style based features              | Completed                     |
| 2 | Text -word- based pred features - predictions of Multinonial Naive Bayes on word based tf-idf| Completed |
| 3 | Text -character -based features - prediction of Multinonial Naive Bayes on based tf-idf | Completed |
| 4 | Text -word_based features - predictions of Multinonial Naive Bayes on word counts vector | Completed |
| 5 | Text -character-based features - prediction of Multinonial Naive Bayes on character counts vector | In Proress |
| 6 | SVD comp on word based tf_idf | Completed |
| 7 | SVD comp on character based tf_idf | Due |
| 8 | **Markov event based features - P(*author/sentence*) for all Authors **| Completed  |
| 9 | Sentiment Analysis based features - flagging sentence as having surprize, fear, jovial feelings | In progress |
| 10 | word embedings based features - sentence to vector using conventional approach  | To be started |




# Preamble 
In this challange, data science community is challenged to predict the author of excerpts from horror stories by Edgar Allan Poe, Mary Shelley, and HP Lovecraft. This kernel will be my attempt on topic modelling and creating a multiclass classifier. I will be covering different feature extraction techniques- handcrafted and using tf-idf etc, will be explaining the nadir of topic modelling- LDA, and NNMF. Further Topic I will add as we go ahead and analysis the data. So, Lets dive in ..


In [ ]:
# Importing the libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import time
import spacy
from scipy.misc import imread


In [ ]:
start_time = time.time()
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
end = time.time()
print("Time taken in reading the input files is {}.".format(end - start_time))
train_df.head()

We are given the minimum number of fields here - 
- **id** -  Id of the record 
- **text** - Text - main input data 
- **author** - Author of the given text
In train data the author is given for each text for training, and in text we are only given text data and we are expected to predict the author using the given text. **

In [ ]:
# Explore the dataset 
print("Number of rows in train dataset {} ".format(train_df.shape[0]))
print("Number of rows in test dataset {} ".format(test_df.shape[0]))
print("No duplicates in train data") if train_df.shape[0] == train_df.text.unique().__len__() else print("Oops")
print("No duplicates in test data") if test_df.shape[0] == test_df.text.unique().__len__() else print("Oops")
print("Unique author is data {}.".format(train_df.author.unique()))
print("Number of nulls in the train is {} and text is {}.".format(train_df.isnull().sum().sum(), test_df.isnull().sum().sum()))

In [ ]:
# Checking the frequencies of text for all author in training dataset
author_summary = pd.DataFrame(train_df.groupby('author')['text'].count())
author_summary.reset_index(inplace = True)
plt.figure(figsize=(10,6))
sns.barplot(author_summary.author, author_summary.text)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Author Name', fontsize=12)
plt.show()

In [ ]:
# Lets check the avg number of words in text written by each author in a histogram 
def word_count(row):
    """function to calculate the count of words in a given text """
    text = row['text']
    text_splited = text.split(' ')
    word_count = text_splited.__len__()
    return word_count

train_df['word_count'] = ''
train_df['word_count'] = train_df.apply(lambda row: word_count(row), axis =1)
test_df['word_count'] = test_df.apply(lambda row: word_count(row), axis =1)

In [ ]:
start = time.time()
fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True, figsize=(14,10))
sns.distplot(train_df.loc[train_df['author']=='EAP']['word_count'].values, ax=ax[0], color='blue', label='Edgar Allan Poe')
sns.distplot(train_df.loc[train_df['author']=='MWS']['word_count'].values, ax=ax[1], color='green', label='Mary Shelley')
sns.distplot(train_df.loc[train_df['author']=='HPL']['word_count'].values, ax=ax[2], color='red', label='HP Lovecraft')
ax[0].legend(loc=0)
ax[1].legend(loc=0)
ax[2].legend(loc=0)
plt.show()
end = time.time()
print("Time taken by above cell is {}.".format((end-start)))

**Findings** - It can be clearly seen that most of the texts given in the training dataset are has word count from 0 to ~100, but the distribution is diferent in right portion of histograms.  HP lovecrafts's text are less scattered, followed by EAP's, and Mary Sheley's text few texts has length ~900 too. This information can be used in hand designed features, for example we can make feature of 0/1 if text is greater than 250 or not, or we can make bins and use this feature, this will sure help in classification of Mary Shelley.

# Fun time- Wordclouds
If you go in consulting, we see the beautiful word_clouds, expressing the words in horizontal and vertical fashion.The word cloud is nothing but a representation of words in arrangement, in which the size of the word is proportional to the frequency of its occurance. Lets me please your eyes by ploting different wordclouds for different authors. 

I have learned from anistropic that masked wordclouds look much better than nornmal one, So, I am also making wordclouds with different masks. Idea and embeddings are taken from anistropic's notebook, thanks anistropic.


In [ ]:
eap_64 = b'...'

In [ ]:
hpl_64 = b'...'

In [ ]:
mws_64 = b'...'

In [ ]:
import codecs
# Generate the Mask for EAP
f1 = open("eap.png", "wb")
f1.write(codecs.decode(eap_64,'base64'))
f1.close()
img1 = imread("eap.png")
# img = img.resize((980,1080))
hcmask = img1

f2 = open("mws.png", "wb")
f2.write(codecs.decode(mws_64,'base64'))
f2.close()
img2 = imread("mws.png")
hcmask2 = img2

f3 = open("hpl.png", "wb")
f3.write(codecs.decode(hpl_64,'base64'))
f3.close()
img3 = imread("hpl.png")
hcmask3 = img3

In [ ]:
from wordcloud import WordCloud, STOPWORDS
eap = train_df[train_df.author=="EAP"]["text"].values
hpl = train_df[train_df.author=="HPL"]["text"].values
mws = train_df[train_df.author=="MWS"]["text"].values

wc = WordCloud(background_color="white", max_words=5000, 
               stopwords=STOPWORDS, max_font_size= 50)
# generate word cloud
wc.generate(" ".join(train_df.text.values))

# show
plt.figure(figsize=(16,13))
plt.imshow(wc, interpolation='bilinear')
plt.title("words from all author", fontsize=14,color='seagreen')
plt.axis("off")

In [ ]:
wc = WordCloud(background_color="white", max_words=5000, mask = hcmask2,
               stopwords=STOPWORDS, max_font_size= 50)
# generate word cloud
wc.generate(" ".join(mws))

# show
plt.figure(figsize=(16,13))
plt.imshow(wc, interpolation='bilinear')
plt.title("Mary Shelley", fontsize=14,color='red')
plt.axis("off")

In [ ]:
wc = WordCloud(background_color="white", max_words=5000, mask = hcmask,
               stopwords=STOPWORDS, max_font_size= 45)
# generate word cloud
wc.generate(" ".join(eap))

# show
plt.figure(figsize=(16,13))
plt.imshow(wc, interpolation='bilinear')
plt.title("EAP", fontsize=14,color='seagreen')
plt.axis("off")

In [ ]:
wc = WordCloud(background_color="white", max_words=5000, mask = hcmask3,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc.generate(" ".join(hpl))

# show
plt.figure(figsize=(16,13))
plt.imshow(wc, interpolation='bilinear')
plt.title("HPL", fontsize=14,color='white')
plt.axis("off")

# Animation - Dancing pennywise .. on the ting go pop, pop, pop, pop...

In [ ]:
# first of all we download 4 masks of dancing positions of pennywise ...
mask_el = b'...'
mask_l = b'...'
mask_r = b'...'
mask_er = b'...'


In [ ]:
start = time.time()
wc_list=[]

pw = open("p_el.png", "wb")
pw.write(codecs.decode(mask_el,'base64'))
pw.close()
pw_el = imread("p_el.png")
el = pw_el
wc1 = WordCloud(background_color="white", max_words=5000, mask = el,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc1.generate(" ".join(mws))
wc_list.append(wc1)

pw2 = open("p_l.png", "wb")
pw2.write(codecs.decode(mask_l,'base64'))
pw2.close()
pw_l = imread("p_l.png")
l = pw_l
wc2 = WordCloud(background_color="white", max_words=5000, mask = l,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc2.generate(" ".join(eap))
wc_list.append(wc2)

pw3 = open("p_r.png", "wb")
pw3.write(codecs.decode(mask_r,'base64'))
pw3.close()
pw_r = imread("p_r.png")
r = pw_r
wc3 = WordCloud(background_color="white", max_words=5000, mask = r,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc3.generate(" ".join(hpl))
wc_list.append(wc3)

pw4 = open("p_er.png", "wb")
pw4.write(codecs.decode(mask_er,'base64'))
pw4.close()
pw_er = imread("p_er.png")
er = pw_er
wc4 = WordCloud(background_color="white", max_words=5000, mask = er,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc4.generate(" ".join(train_df.text.values))
wc_list.append(wc4)

pw3 = open("p_r.png", "wb")
pw3.write(codecs.decode(mask_r,'base64'))
pw3.close()
pw_r = imread("p_r.png")
r = pw_r
wc3 = WordCloud(background_color="white", max_words=5000, mask = r,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc3.generate(" ".join(hpl))
wc_list.append(wc3)

pw2 = open("p_l.png", "wb")
pw2.write(codecs.decode(mask_l,'base64'))
pw2.close()
pw_l = imread("p_l.png")
l = pw_l
wc2 = WordCloud(background_color="white", max_words=5000, mask = l,
               stopwords=STOPWORDS, max_font_size= 35)
# generate word cloud
wc2.generate(" ".join(eap))
wc_list.append(wc2)

In [ ]:
from IPython.display import HTML
from matplotlib.pyplot import *
from matplotlib import cm
from matplotlib import animation
import io
import base64
# imports related to animation

In [ ]:
start = time.time()
def build_gif(imgs = wc_list, show_gif=False, save_gif=True, title=''):
    """function to create a gif of heatmaps"""
    fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(21,15))
    ax.set_axis_off()
    author_range = ["Because they pop..", "pop! pop! pop!", "skraaa pop! pop! pop!", "pop! pop! pop! pop! pop!", "skraaa pop! pop! pop!","pop! pop! pop!"]
    def show_im(pairs):
        ax.clear()
        ax.set_title(str((pairs[0])))
        ax.imshow(pairs[1])
        ax.set_axis_off() 
    pairs = list(zip(author_range, imgs))
    im_ani = animation.FuncAnimation(fig, show_im, pairs,interval=350, repeat_delay=0, blit=False)
    plt.cla()
    if save_gif:
        im_ani.save('animation.gif', writer='imagemagick') #, writer='imagemagick'
    if show_gif:
        plt.show()
    return
end = time.time()
print("Time taken by above cell is {}".format(end-start))

In [ ]:
start = time.time()
build_gif()
end = time.time()
print(end-start)

In [ ]:
filename = 'animation.gif'
video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))


# Fun time over, Now, I will seriously focus on feature engineering..
Here we can go ahead with finding features based on the text like have the frequency data each of the word and then apply NB ( like they do in spam and ham classification) or extract features that may be the characteristics of different author and then using those features along with the text releted features and train some classifier on it.

## Features corresponding to characteristics of different author's writting style
Here my assuption is that each text that's been given is a paragraph in one of the books of corresponding author. Based on that assumption if we try and we can see if there is any pattern in author's writting style and based on that we can classify which text belong to which author. As of now, I am thinking about following features that can be put in this category. they are - 
- word_count - The count of words in given text 
- unique_words_fraction - Fraction of words that are unique in a given text 
- stop_words - Fraction of stopwords present in a given text - Number of stopwords/Total words
- punctuations - Fraction of punctuation present in a given text - Number of puctuations/Total words
- fract_Nouns - Fraction of Nuons
- fract_adj - Fraction of Adjectives present in a text 

As of now I can think of these aforesaid features, will update this list of features if I extract any other features than mentioned features. Lets define function for features releted to author's characteristics and then run those function for final feature extraction.

**Note** - Here for some or most of the features I have used fraction and not the number as my person experience of document classification using manualluy designed features says that it make more sense to use fraction as if one text is larger thgan the author the number will mislead you while fraction will cover the pith about author's characteristics.



In [ ]:
train_df.head()

In [ ]:
import string
def unique_word_fraction(row):
    """function to calculate the fraction of unique words on total words of the text"""
    text = row['text']
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    unique_count = list(set(text_splited)).__len__()
    return (unique_count/word_count)


eng_stopwords = set(stopwords.words("english"))
def stopwords_count(row):
    """ Number of stopwords fraction in a text"""
    text = row['text'].lower()
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    stopwords_count = len([w for w in text_splited if w in eng_stopwords])
    return (stopwords_count/word_count)


def punctuations_fraction(row):
    """functiopn to claculate the fraction of punctuations over total number of characters for a given text """
    text = row['text']
    char_count = len(text)
    punctuation_count = len([c for c in text if c in string.punctuation])
    return (punctuation_count/char_count)


def char_count(row):
    """function to return number of chracters """
    return len(row['text'])

def fraction_noun(row):
    """function to give us fraction of noun over total words """
    text = row['text']
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    pos_list = nltk.pos_tag(text_splited)
    noun_count = len([w for w in pos_list if w[1] in ('NN','NNP','NNPS','NNS')])
    return (noun_count/word_count)

def fraction_adj(row):
    """function to give us fraction of adjectives over total words in given text"""
    text = row['text']
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    pos_list = nltk.pos_tag(text_splited)
    adj_count = len([w for w in pos_list if w[1] in ('JJ','JJR','JJS')])
    return (adj_count/word_count)

def fraction_verbs(row):
    """function to give us fraction of verbs over total words in given text"""
    text = row['text']
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    pos_list = nltk.pos_tag(text_splited)
    verbs_count = len([w for w in pos_list if w[1] in ('VB','VBD','VBG','VBN','VBP','VBZ')])
    return (verbs_count/word_count)



# Features extraction
### Applying functions-defined above-to extract features based on characteristics of author's writing style...

In [ ]:
train_df['unique_word_fraction'] = train_df.apply(lambda row: unique_word_fraction(row), axis =1)
train_df['stopwords_count'] = train_df.apply(lambda row: stopwords_count(row), axis =1)
train_df['punctuations_fraction'] = train_df.apply(lambda row: punctuations_fraction(row), axis =1)
train_df['char_count'] = train_df.apply(lambda row: char_count(row), axis =1)
train_df['fraction_noun'] = train_df.apply(lambda row: fraction_noun(row), axis =1)
train_df['fraction_adj'] = train_df.apply(lambda row: fraction_adj(row), axis =1)
train_df['fraction_verbs'] = train_df.apply(lambda row: fraction_verbs(row), axis =1)
train_df.head()

In [ ]:
test_df['unique_word_fraction'] = test_df.apply(lambda row: unique_word_fraction(row), axis =1)
test_df['stopwords_count'] = test_df.apply(lambda row: stopwords_count(row), axis =1)
test_df['punctuations_fraction'] = test_df.apply(lambda row: punctuations_fraction(row), axis =1)
test_df['char_count'] = test_df.apply(lambda row: char_count(row), axis =1)
test_df['fraction_noun'] = test_df.apply(lambda row: fraction_noun(row), axis =1)
test_df['fraction_adj'] = test_df.apply(lambda row: fraction_adj(row), axis =1)
test_df['fraction_verbs'] = test_df.apply(lambda row: fraction_verbs(row), axis =1)
test_df.head()

# Features (characteristics of author's writing style) visualization 
When we extract these features we know already that these features doen't have very large predictive power (by experience), but can help in some cases, so, Lets examine how these features are behaving for this problem. We will use boxplot and voilinplots for visualizing these features. 

In [ ]:
f, axes = plt.subplots(3,2,figsize=(21, 18), sharex=True, sharey = False)
sns.despine(left=True)
sns.set_context("poster")
sns.violinplot(x="author", y="unique_word_fraction", data=train_df, ax = axes[0,0])
sns.violinplot(x="author", y="stopwords_count", data=train_df, ax = axes[0,1])
sns.violinplot(x="author", y="punctuations_fraction", data=train_df, ax = axes[1,0])
sns.violinplot(x="author", y="fraction_verbs", data=train_df, ax = axes[1,1])
sns.violinplot(x="author", y="fraction_noun", data=train_df, ax = axes[2,0])
sns.violinplot(x="author", y="fraction_adj", data=train_df, ax = axes[2,1])
sns.despine(left=True)

**Findings -** 
- **Stopwords**
 - All the authors are have similar pattern for stopwords, but HPL is using slightly less number of stopwords per text. Somewhat useful
- **Punctuation_fraction**
 - See the grapth of punctuationfraction is very smooth here, for all the three authors. This is very useful as you can just classify cases sometimes based on this functions value. for example a decision tree can use this feature for spliting and claerly assign EAP if the value of punctuation fraction is greater than 0.3. so this feature will sure help in tree split in XGB.
 - Secondly, I haven't reproduced it here but if you see the plot of number of punctuations in here-  (http://)https://www.kaggle.com/sudalairajkumar/simple-feature-engg-notebook-spooky-author in SRK's notebook, you will see that its not smooth. So, normalization here will sure play a part in final outcome.
 - Most of the text have similar mean and have similar quartile ranges, so, this doesn't contain very high prediction power but it will sure help classifying some cases.
- **Other Features**
 - **Unique_words** - HPL is using more unique words than the other two.
 - **Noun,verbs, adj** - HPL is using slightly more number of nouns than other two.
 

In [ ]:
# word_count and char_counts
f, axes = plt.subplots(nrows=1, ncols=2,figsize=(24, 11), sharex=True, sharey = False)
sns.set(style="ticks")
sns.set_context("poster")
sns.boxplot(y="word_count", x="author", data=train_df, palette="PRGn", ax = axes[0])
sns.boxplot(y="char_count", x="author", data=train_df, palette="PRGn", ax = axes[1])
sns.despine(offset=10, trim=True)

# Extracting sentiments in sentences 
I will use four different kinds of sentiments and using the most frequently occurring words to that sentiment, will given if that particular sentiment is present or not. The four different kinds of sentiments are as follows - 
- Surprize - 1/0 if it contains surprize words
- Fear -1/0 using fear words
- Joy - 1/0 using jovial words



In [ ]:
Surprize_words = ['finally', 'suddenly', 'mystery', 'fortune', 'sweet', 'horror', 'chance', '']

# Extracting Features related to text
## Tf-idf features 

In [ ]:
start = time.time()
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())
end = time.time()
print("Time taken in tf-idf is {}.".format(end-start))

cols_to_drop = ['id', 'text']
train_X = train_df.drop(cols_to_drop+['author'], axis=1)
test_X = test_df.drop(cols_to_drop, axis=1)
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)

In [ ]:
# this function is taken from SRK's notebook
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
train_df["nb_cvec_eap"] = pred_train[:,0]
train_df["nb_cvec_hpl"] = pred_train[:,1]
train_df["nb_cvec_mws"] = pred_train[:,2]
test_df["nb_cvec_eap"] = pred_full_test[:,0]
test_df["nb_cvec_hpl"] = pred_full_test[:,1]
test_df["nb_cvec_mws"] = pred_full_test[:,2]

In [ ]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
train_df["nb_cvec_eap"] = pred_train[:,0]
train_df["nb_cvec_hpl"] = pred_train[:,1]
train_df["nb_cvec_mws"] = pred_train[:,2]
test_df["nb_cvec_eap"] = pred_full_test[:,0]
test_df["nb_cvec_hpl"] = pred_full_test[:,1]
test_df["nb_cvec_mws"] = pred_full_test[:,2]

In [ ]:
start = time.time()
n_comp = 400
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
train_df = pd.concat([train_df, train_svd], axis=1)
test_df = pd.concat([test_df, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd
end = time.time()
print("Time taken in tf-idf is {}.".format(end-start))

In [ ]:
#character based tfidf
start = time.time()
tfidf_vec = TfidfVectorizer(ngram_range=(1,5), analyzer='char')
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())
n_comp = 40
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))   
train_svd.columns = ['svd_char_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_char_'+str(i) for i in range(n_comp)]
train_df = pd.concat([train_df, train_svd], axis=1)
test_df = pd.concat([test_df, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd
end = time.time()
print("Time taken in tf-idf is {}.".format(end-start))

# Markov Model based features 

In [ ]:
val_df = train_df.sample(frac=0.2, replace=False)
trn_df = train_df.drop(val_df.index)
print(trn_df.shape[0])
print(val_df.shape[0])
print(train_df.shape[0])
trn_df.head()

In [ ]:
import random

class Dictogram(dict):
    def __init__(self, iterable=None):
        """Initialize this histogram as a new dict; update with given items"""
        super(Dictogram, self).__init__()
        self.types = 0  # the number of distinct item types in this histogram
        self.tokens = 0  # the total count of all item tokens in this histogram
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        """Update this histogram with the items in the given iterable"""
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        """Return the count of the given item in this histogram, or 0"""
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        # Another way:  Should test: random.choice(histogram.keys())
        random_key = random.sample(self, 1)
        return random_key[0]

    def return_weighted_random_word(self):
        # Step 1: Generate random number between 0 and total count - 1
        random_int = random.randint(0, self.tokens-1)
        index = 0
        list_of_keys = self.keys()
        # print 'the random index is:', random_int
        for i in range(0, self.types):
            index += self[list_of_keys[i]]
            # print index
            if(index > random_int):
                # print list_of_keys[i]
                return list_of_keys[i]

In [ ]:
# markov chain based features, 3 words memory 
def make_higher_order_markov_model(order, data):
    markov_model = dict()

    for i in range(0, len(data)-order):
        # Create the window
        window = tuple(data[i: i+order])
        # Add to the dictionary
        if window in markov_model:
            # We have to just append to the existing Dictogram
            markov_model[window].update([data[i+order]])
        else:
            markov_model[window] = Dictogram([data[i+order]])
    return markov_model

In [ ]:
print(trn_df.loc[train_df['author']=='EAP'].shape[0])
def tokenixed_list(row):
    """function to calculate the fraction of unique words on total words of the text"""
    text = row['text']
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    return (text_splited)
trn_df['splited_text'] = trn_df.apply(lambda row: tokenixed_list(row), axis =1)

In [ ]:
eap = trn_df.loc[trn_df['author']=='EAP']['splited_text'].values
flat_eap = []
eap_flat = [flat_eap.extend(x) for x in eap]
flat_eap.__len__()
eap_MM = make_higher_order_markov_model(3, flat_eap)

In [ ]:
hpl = trn_df.loc[trn_df['author']=='HPL']['splited_text'].values
flat_hpl = []
hpl_flat = [flat_hpl.extend(x) for x in hpl]
flat_hpl.__len__()
hpl_MM = make_higher_order_markov_model(3, flat_hpl)

In [ ]:
ms = trn_df.loc[trn_df['author']=='MWS']['splited_text'].values
flat_ms = []
ms_flat = [flat_ms.extend(x) for x in ms]
flat_ms.__len__()
ms_MM = make_higher_order_markov_model(3, flat_ms)

In [ ]:
#probablity calculation using markov chain
#print(eap_MM[('no', 'means', 'of')])
#print(sum([x for x in eap_MM[('no', 'means', 'of')].values()]))
def spit_out_prob(tupl , word , mm1= eap_MM, mm2 = hpl_MM, mm3 = ms_MM):
    """function to spit out a prob of a word given a tuple and a word and a markov model"""
    """fix this function, its working but not calculating probabilities, but providing
    some confidance index, greater than confidance index is, larger is the prob"""
    try:
        a = mm1[tupl][word]
    except:
        a = 0 
    try:
        b = mm2[tupl][word]
    except:
        b = 0
    try:
        c = mm3[tupl][word]
    except:
        c = 0
    try:
        prob_word = a/(a+b+c)#(sum([x for x in mm[tupl].values()]))
        return (prob_word)
    except:
        return (0) #earlier it was 1



def make_tupl(sentence =trn_df.loc[train_df['author']=='EAP']['splited_text'][0] , n=3):
    """function to make tuples of n size given a sentence and n"""
    list_of_tuple = []
    word_1 = []
    for i in list(range(sentence.__len__()-3)):
        tuple_1 = (sentence[i], sentence[i+1], sentence[i+2])
        list_of_tuple.append(tuple_1)
        word_1.append(sentence[i+3])
    return (list_of_tuple, word_1)


def sent_to_prob_eap(sentence, mm1 = eap_MM, p_eap =1/3, mm2 = hpl_MM, mm3 = ms_MM):
    """function to get the markov model to give prob of a author given a sentence """
    list_of_tuples = make_tupl(sentence, n =3)[0]
    words = make_tupl(sentence, n =3)[1]
    #print(list_of_tuples, words)
    #p = p_eap 
    p=0
    for i in list(range(words.__len__())):
        #print(list_of_tuples[i], words[i])
        p = p+spit_out_prob(list_of_tuples[i], words[i], mm1 =eap_MM, mm2 =hpl_MM, mm3 =ms_MM)
    return(p)

def sent_to_prob_hpl(sentence, mm2 = hpl_MM, p_hpl =1/3, mm1 = eap_MM, mm3 = ms_MM):
    """function to get the markov model to give prob of a author given a sentence """
    list_of_tuples = make_tupl(sentence, n =3)[0]
    words = make_tupl(sentence, n =3)[1]
    #print(list_of_tuples, words)
    #p = p_hpl 
    p = 0
    for i in list(range(words.__len__())):
        #print(list_of_tuples[i], words[i])
        p = p+spit_out_prob(list_of_tuples[i], words[i], mm1 =hpl_MM, mm2=eap_MM, mm3=ms_MM)
    return(p)

def sent_to_prob_ms(sentence, mm3 = ms_MM, p_ms =1/3, mm2 = hpl_MM, mm1 = eap_MM):
    """function to get the markov model to give prob of a author given a sentence """
    list_of_tuples = make_tupl(sentence, n =3)[0]
    words = make_tupl(sentence, n =3)[1]
    #print(list_of_tuples, words)
    #p = p_ms 
    p = 0
    for i in list(range(words.__len__())):
        #print(list_of_tuples[i], words[i])
        p = p+spit_out_prob(list_of_tuples[i], words[i], ms_MM, hpl_MM, eap_MM)
    return(p)



In [ ]:
# calculating markov chain based features
trn_df['EAP_markov_3'] = trn_df['splited_text'].apply(lambda row: sent_to_prob_eap(sentence =row))
trn_df['HPL_markov_3'] = trn_df['splited_text'].apply(lambda row: sent_to_prob_hpl(sentence = row))
trn_df['MWS_markov_3'] = trn_df['splited_text'].apply(lambda row: sent_to_prob_ms(sentence = row))
trn_df[['MWS_markov_3', 'EAP_markov_3', 'HPL_markov_3', 'author']].head()
del trn_df['splited_text']

In [ ]:
val_df['splited_text'] = val_df.apply(lambda row: tokenixed_list(row), axis =1)
val_df['EAP_markov_3'] = val_df['splited_text'].apply(lambda row: sent_to_prob_eap(sentence =row))
val_df['HPL_markov_3'] = val_df['splited_text'].apply(lambda row: sent_to_prob_hpl(sentence = row))
val_df['MWS_markov_3'] = val_df['splited_text'].apply(lambda row: sent_to_prob_ms(sentence = row))
val_df1 = val_df[['MWS_markov_3', 'EAP_markov_3', 'HPL_markov_3','author']].copy()
del val_df['splited_text']

In [ ]:
test_df['splited_text'] = test_df.apply(lambda row: tokenixed_list(row), axis =1)
test_df['EAP_markov_3'] = test_df['splited_text'].apply(lambda row: sent_to_prob_eap(sentence =row))
test_df['HPL_markov_3'] = test_df['splited_text'].apply(lambda row: sent_to_prob_hpl(sentence = row))
test_df['MWS_markov_3'] = test_df['splited_text'].apply(lambda row: sent_to_prob_ms(sentence = row))
test_df1 = test_df[['MWS_markov_3', 'EAP_markov_3', 'HPL_markov_3']].copy()
del test_df['splited_text']

# XGB Model

In [ ]:
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
trn_y = trn_df['author'].map(author_mapping_dict)
val_y = val_df['author'].map(author_mapping_dict)
cols_drop = ['id', 'text','MWS_markov_3', 'EAP_markov_3', 'HPL_markov_3']
trn_X = trn_df.drop(cols_drop+['author'], axis=1)
val_X = val_df.drop(cols_drop+['author'], axis=1)
test_X = test_df.drop(cols_drop, axis=1)

In [ ]:
print(52*'-')
print("shape of train val is {} and {}, test is {}".format(trn_X.shape[1], val_X.shape[1], test_X.shape[1]))
print(52*'-')

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
Xtr = trn_X.values
Xv = val_X.values
ytr = trn_y
yv = val_y
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test_X.values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

start = time.time()
xgb_par = {'min_child_weight': 1, 'eta': 0.1, 'colsample_bytree': 0.7, 'max_depth': 3,
            'subsample': 0.8, 'lambda': 2.0, 'nthread': -1, 'silent': 1,
            'eval_metric': "mlogloss", 'objective': 'multi:softprob', 'num_class': 3}

model_1 = xgb.train(xgb_par, dtrain, 1000, watchlist, early_stopping_rounds=50, maximize=False, verbose_eval=40)
print('Modeling RMSLE %.5f' % model_1.best_score)
end = time.time()
print("Time taken in training is {}.".format(end - start))

In [ ]:
print(test_X.shape, trn_X.shape)

In [ ]:

start = time.time()
yvalid = model_1.predict(dvalid)
ytest = model_1.predict(dtest)
end = time.time()
print("Time taken in prediction is {}.".format(end - start))

In [ ]:
val_df1.head()
val_df2 = pd.DataFrame(yvalid)
val_df2.head()
val_df2.columns = ['EAP', 'HPL', 'MWS']
val_df2.head()
val_df1.reset_index(inplace = True, drop =True)
val_df1.head()


In [ ]:
val = pd.concat([val_df1, val_df2], axis =1)
val.head()

In [ ]:
start = time.time()
if test_df.shape[0] == ytest.shape[0]:
    print('Test shape OK.') 
test_id = test_df['id'].values
out_df = pd.DataFrame(ytest)
out_df.columns = ['EAP', 'HPL', 'MWS']
out_df.insert(0, 'id', test_id)
out_df.to_csv("Mahesh_xgb_test.csv", index=False)
end = time.time()
print("Time taken in training is {}.".format(end - start))

In [ ]:
## Let's submit and see where we stand on LB 
out_df.head()
test_df2 = out_df[['EAP','HPL', 'MWS']]
test_df2.head()
test_df1.head()
t = pd.concat([test_df1, test_df2], axis =1)
t.head()

In [ ]:
# linear combination 
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
trn_y = val['author'].map(author_mapping_dict)
import xgboost as xgb
from sklearn.model_selection import train_test_split
Xtr = val[['MWS_markov_3','EAP_markov_3','HPL_markov_3','EAP','HPL','MWS']].values
ytr = trn_y
dtrain = xgb.DMatrix(Xtr, label=ytr)
#dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(t.values)
watchlist = [(dtrain, 'train')]

start = time.time()
xgb_par = {'min_child_weight': 1, 'eta': 0.1, 'colsample_bytree': 0.7, 'max_depth': 3,
            'subsample': 0.8, 'lambda': 2.0, 'nthread': -1, 'silent': 1,
            'eval_metric': "mlogloss", 'objective': 'multi:softprob', 'num_class': 3}

model_2 = xgb.train(xgb_par, dtrain, 2000, watchlist, early_stopping_rounds=50, maximize=False, verbose_eval=100)
print('Modeling RMSLE %.5f' % model_2.best_score)
end = time.time()
print("Time taken in training is {}.".format(end - start))

In [ ]:
start = time.time()
ytest = model_2.predict(dtest)
end = time.time()
print("Time taken in prediction is {}.".format(end - start))

In [ ]:
start = time.time()
if test_df.shape[0] == ytest.shape[0]:
    print('Test shape OK.') 
test_id = test_df['id'].values
out_df = pd.DataFrame(ytest)
out_df.columns = ['EAP', 'HPL', 'MWS']
out_df.insert(0, 'id', test_id)
out_df.to_csv("Mahesh_xgb_test_lc.csv", index=False)
end = time.time()
print("Time taken in training is {}.".format(end - start))

In [ ]:
out_df.head()

**Upvote if you found this kernel useful ..Thanks **